# **Prova Final: Computação Científica II - 2023**
##### **Data**: 30/11/2023 - 03/12/2023
##### **Alune**: Igor Lima Strozzi
##### **DRE**: 112040449

**Observação**: vou escrever em inglês, estou mais acostumado.

## Questão 1


Temos o problema de contorno definido por, para $ (x, y) \in (0, 1)^2 $ ,
\begin{gather}
	-\nabla(\kappa\nabla T) = 0 \texttt{;} \\
	T(x, 0) = T(x, 1) = a \texttt{;} \\
	T(0, y) = b \texttt{;} \\
	-\kappa \frac{\partial T}{\partial x}(1, y) = h(T - T_{\text{out}})\texttt{.}
\end{gather}

É imediato observar que, de $(1)$ acima, temos $-\left( \frac{\partial}{\partial x}\left(\kappa \frac{\partial T}{\partial x}\right) + \frac{\partial}{\partial y}\left(\kappa \frac{\partial T}{\partial y}\right) \right) = - \left(\left( \kappa \frac{\partial^2 T}{\partial x^2}\right) + \left( \kappa \frac{\partial^2 T}{\partial y^2}\right)\right)$, portanto (sendo $\kappa > 0$), $b^2 - ac = 0^2 - (-\kappa)^2 < 0$, e portanto a equação $(1)$ é uma equação diferencial parcial elíptica.


In [2]:
%load_ext itikz

In [3]:
%%itikz
\documentclass[tikz]{standalone}
\begin{document}
\begin{tikzpicture}
\draw[help lines] grid (5, 5);
\draw[fill=black] (1, 1) rectangle (2, 2);
\draw[fill=black] (2, 1) rectangle (3, 2);
\draw[fill=black] (3, 1) rectangle (4, 2);
\draw[fill=black] (3, 2) rectangle (4, 3);
\draw[fill=black] (2, 3) rectangle (3, 4);
\end{tikzpicture}
\end{document}

FileNotFoundError: [WinError 2] The system cannot find the file specified

%%itikz
\documentclass[tikz]{standalone}
\begin{document}
	\begin{tikzpicture}
		% Draw a 3x4 grid with dashed lines
		\draw[step=1cm,gray,very thin, dashed] (0,0) grid (3,4);
		
		% Draw the boundary of the grid
		\draw[thick] (0,0) -- (0,4) -- (3,4) -- (3,0) -- cycle;
		
		% Place dots at each intersection of the grid
		\foreach \x in {0,1,2,3}
		\foreach \y in {0,1,2,3,4}
		\filldraw[black] (\x,\y) circle (2pt);
		
		% Add labels next to the grid
		\node at (-0.5, 2) {$b$};
		\node at (1.5, 4.5) {$a$};
		\node at (1.5, -0.5) {$a$};
		\node[align=center] at (4.8, 2) {$-\kappa \frac{\partial T}{\partial x}$ \\ $= h(T - T_{\text{out}})$};
	\end{tikzpicture}
\end{document}